# feature engineering
Start:
* one database with two collections that hold all the sprints and issues with all the raw data
* only addition to this raw data is a field which repository they belong to

End:

* Sprints:
  * Make aggregation of each sprint's metadata:
    * sprint planned duration in days
    * sprint overdue boolean (completedDate > endDate)
    * Number of issues inside sprint
    * Median number of issues with changed sprint
* Issues:
  * predictor: has sprint field in change log (boolean)
  * Number of sprint fields in change log (int)
  * Add originSprintId field (uuid)
  * Add above sprint metadata fields to every issue
  * All not used (custom)fields should be removed

In [1]:
# Auxiliary
import pymongo
import re
import json
import numpy as np
from datetime import datetime
from dateutil import parser

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']

In [2]:
# Issues
coll_issues = db['01_Issues']
issues = coll_issues.find()

# Number of all and sprint-only fields in change log (int)
for issue in issues:
    sprint_changes = 0
    total_changes = issue['changelog']['total']
    histories = issue['changelog']['histories']
    for history in histories:
        for item in history['items']:
            if 'sprint' in item['field'].lower():
                sprint_changes += 1

    coll_issues.update_one({"id": issue['id']}, {"$set": {"totalChanges": total_changes, "sprintChanges": sprint_changes}})


In [7]:
# Sprints
coll_sprints = db['02_Sprints']
sprints = coll_sprints.find()

for sprint in sprints:
    # sprint planned duration in days
    try:
        startDate = parser.parse(sprint['startDate'])
        endDate = parser.parse(sprint['endDate'])
        plannedDuration =  (endDate - startDate).days
    except:
        plannedDuration = np.nan

    # Number of issues inside sprint
    issues = sprint['issues']
    noOfIssues = len(issues)
    changes_in_issues = 0
    sprint_changes_in_issues = 0
    
    # Get total number of changes and sprint changes 
    for issue in issues:
        print(issue)
        doc = coll_issues.find_one({"id": issue})
        changes_in_issues += doc['totalChanges']
        sprint_changes_in_issues += doc['sprintChanges']

    coll_sprints.update_one({'id': sprint['id']}, {'$set': {'plannedDuration': plannedDuration, 'noOfIssues': noOfIssues, "totalChanges": changes_in_issues, "sprintChanges": sprint_changes_in_issues}})



Jira467709
Jira473042
Jira472411
Jira526575
Jira518198
Jira427310
Jira838593
Jira833916
Jira833915
Jira787757
Jira790384
Jira790275
Jira787557
Jira658115
Jira606508
Jira794535
Jira792956
Jira766952
Jira752904
Jira743691
Jira773287
Jira750601
Jira772584
Jira826298
Jira672999
Jira645134
Jira831949
Jira783339
Jira788971
Jira853957
Jira845406
Jira841317
Jira838797
Jira814675
Jira814674
Jira670364
Jira667359
Jira854863
Jira850917
Jira847847
Jira843593
Jira825961
Jira793877
Jira736064
Jira726063
Jira731419
Jira735320
Jira740785
Jira743970
Jira674039
Jira661172
Jira774227
Jira654325
Jira651014
Jira642039
Jira602924
Jira453895
Jira419905
Jira415602
Jira647773
Jira645572
Jira637544
Jira629658
Jira592320
Jira619915
Jira643863
Jira639827
Jira636394
Jira633302
Jira631247
Jira630843
Jira630442
Jira630056
Jira595429
Jira628852
Jira625772
Jira625182
Jira623290
Jira622644
Jira622220
Jira618433
Jira606073
Jira605039
Jira604582
Jira575709
Jira249235
Jira256479
Jira258139
Jira425343
Jira405508
Jira595630

In [ ]:
# Add above sprint metadata fields to every issue

In [ ]:
# Keep only a selection of fields as features
fields_to_keep = ['description', 'summary', 'resolution.name', 'priority.name', 'aggregatetimeoriginalestimate', 'timeestimate', 'assignnee.key', 'status.name', 'votes.votes', 'issuetype.name', 'project.id', 'watches.watchcount', 'comments.count']

In [ ]:
def get_origin_sprint(issue_id):
    sprint_id = re.search(r'id=(.*?),', issue_id[]).group(1)

In [ ]:
# Sprints that need the new Issue data
# Median number of issues with changed sprint